In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.metrics import roc_curve

# 1. 이미지 특징 벡터 추출 클래스 정의 (ResNet18 사용)
class FeatureExtractor:
    def __init__(self, model_path):
        # 훈련된 ResNet18 모델 불러오기
        self.model = models.resnet18(weights=None)  # 사전 가중치 없이 정의
        self.model.fc = nn.Linear(self.model.fc.in_features, 31)  # 마지막 레이어 설정
        self.model.load_state_dict(torch.load(model_path, map_location='cpu', weights_only=True))  # 가중치 로드
        self.model.fc = nn.Identity() 
        self.model.eval()

    def extract_features(self, image_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)  # 배치 차원 추가
        with torch.no_grad():
            features = self.model(image)
        return features.squeeze().numpy()

# 사용 예시
feature_extractor = FeatureExtractor('fashion_resnet18.pt')
features = feature_extractor.extract_features('data/validation_image_cropped/T_00253_60_popart_W.jpg')
features

In [20]:

import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

class TrainingImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, label_encoder=None, is_training=False):
        self.img_dir = img_dir
        self.transform = transform
        self.img_labels = self.get_image_labels()
        self.is_training = is_training
        
        if self.is_training:
            # 훈련 데이터에서는 LabelEncoder 사용
            self.label_encoder = label_encoder
            self.labels = self.label_encoder.transform([label for _, label in self.img_labels])
        else:
            # 평가/테스트 모드에서는 인코딩 없이 원래 레이블을 유지
            self.labels = [label for _, label in self.img_labels]

    def get_image_labels(self):
        img_labels = []
        for img_file in os.listdir(self.img_dir):
            label = self.extract_label_from_filename(img_file)
            img_labels.append((img_file, label))
        return img_labels

    def extract_label_from_filename(self, filename):
        label = filename.split(".")[0] # Extract label from filename
        return label

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        if self.is_training:
            img_name = self.img_labels[idx][0]
            label = self.labels[idx]
        else:
            img_name, label = self.img_labels[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, label

# 데이터셋과 데이터로더 생성
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = TrainingImageDataset(img_dir='./data/training_image_cropped', transform=transform)
val_dataset = TrainingImageDataset(img_dir='./data/validation_image_cropped', transform=transform)

# 두 데이터셋을 합치기
from torch.utils.data import ConcatDataset

class CustomConcatDataset(ConcatDataset):
    def __init__(self, datasets):
        super().__init__(datasets)
        self.labels = []
        for dataset in datasets:
            self.labels.extend(dataset.labels)

concat_dataset = CustomConcatDataset([train_dataset, val_dataset])
data_loader = DataLoader(concat_dataset, batch_size=256, shuffle=False)

# 1. 이미지 특징 벡터 추출 클래스 정의 (ResNet18 사용)
class FeatureExtractor:
    def __init__(self, model_path):
        # 훈련된 ResNet18 모델 불러오기
        self.model = models.resnet18(weights=None)  # 사전 가중치 없이 정의
        self.model.fc = nn.Linear(self.model.fc.in_features, 31)  # 마지막 레이어 설정
        self.model.load_state_dict(torch.load(model_path, map_location='cpu', weights_only=True))  # 가중치 로드
        self.model.fc = nn.Identity()
        self.model.eval()
        self.model.to('cuda')  # GPU로 이동

    def extract_features(self, images):
        with torch.no_grad():
            features = self.model(images)
        return features.cpu().numpy()

model_path = 'fashion_resnet18.pt'
extractor = FeatureExtractor(model_path)

features = []
image_names = []

# 배치 단위로 특징 벡터 추출
for images, paths in tqdm(data_loader):
    images = images.to('cuda')  # GPU로 이동
    batch_features = extractor.extract_features(images)
    features.append(batch_features)
    image_names.extend(paths)

# 모든 특징 벡터를 하나의 NumPy 배열로 변환
features = np.vstack(features)

# 특징 벡터를 데이터프레임으로 변환 (이미지 파일명을 인덱스로 사용)
feature_df = pd.DataFrame(features, index=image_names)



100%|██████████| 20/20 [07:30<00:00, 22.52s/it]


In [21]:
image_names

['W_01752_00_metrosexual_M',
 'W_46417_70_military_W',
 'W_01509_00_metrosexual_M',
 'W_18951_50_feminine_W',
 'W_29485_10_sportivecasual_M',
 'W_01706_90_hiphop_M',
 'W_07048_90_hiphop_M',
 'W_02897_90_hiphop_M',
 'W_10212_60_space_W',
 'W_15390_60_mods_M',
 'W_17108_19_normcore_M',
 'W_15783_70_hippie_M',
 'W_06561_10_sportivecasual_M',
 'T_01322_19_normcore_M',
 'W_14468_10_sportivecasual_W',
 'W_65341_80_powersuit_W',
 'W_04372_10_sportivecasual_M',
 'W_15576_70_hippie_M',
 'W_06228_10_sportivecasual_M',
 'W_11096_00_metrosexual_M',
 'W_08845_10_sportivecasual_W',
 'W_02086_19_normcore_W',
 'W_11341_19_normcore_W',
 'W_18894_50_feminine_W',
 'W_29108_90_kitsch_W',
 'W_02378_10_sportivecasual_W',
 'W_15402_70_hippie_M',
 'W_11008_50_ivy_M',
 'W_18966_90_kitsch_W',
 'W_34573_10_sportivecasual_W',
 'W_11015_00_metrosexual_M',
 'W_16423_90_hiphop_M',
 'W_15634_80_bold_M',
 'W_19075_50_classic_W',
 'W_15486_00_metrosexual_M',
 'W_04482_10_sportivecasual_M',
 'W_07959_70_military_W',
 'W

In [23]:
feature_df.to_csv('image_feature_df.csv')

In [41]:
feature_df = pd.read_csv('image_feature_df.csv', index_col=0)
feature_df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
W_01752_00_metrosexual_M,0.000000,0.0,0.0,0.000000,0.000000,9.918851,4.826889,0.0,0.977231,0.0,...,0.378588,3.646782,0.000000,0.0,0.000000,0.0,0.000000,0.079235,0.0,5.655496
W_46417_70_military_W,0.000000,0.0,0.0,0.000000,0.000000,15.640561,0.000000,0.0,2.935284,0.0,...,0.000000,0.000000,0.024593,0.0,0.000000,0.0,0.545377,0.000000,0.0,0.000000
W_01509_00_metrosexual_M,0.000000,0.0,0.0,0.000000,14.170481,8.687386,0.000000,0.0,0.000000,0.0,...,11.313698,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
W_18951_50_feminine_W,0.000000,0.0,0.0,0.522858,0.000000,2.235025,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
W_29485_10_sportivecasual_M,0.000000,0.0,0.0,0.000000,0.000000,2.967141,0.000000,0.0,0.554982,0.0,...,0.000000,0.409312,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T_21988_70_hippie_M,0.000000,0.0,0.0,0.000000,0.000000,10.869076,0.000000,0.0,0.000000,0.0,...,1.326624,0.700061,0.000000,0.0,3.645837,0.0,0.651385,0.015984,0.0,3.218005
W_28022_50_ivy_M,0.000000,0.0,0.0,0.000000,0.000000,1.857327,0.000000,0.0,0.811020,0.0,...,0.000000,0.000000,3.086018,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
W_00551_19_normcore_M,0.000000,0.0,0.0,0.000000,0.000000,17.855057,1.716609,0.0,0.000000,0.0,...,3.368212,4.705262,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.604057
W_03144_50_classic_W,0.611408,0.0,0.0,0.000000,0.000000,1.886823,0.000000,0.0,2.801883,0.0,...,0.000000,0.879312,5.284182,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.060196


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

# 3. 코사인 유사도를 통해 이미지 간 유사도 계산
similarity_matrix = cosine_similarity(feature_df.values)
similarity_df = pd.DataFrame(similarity_matrix, index=image_paths, columns=image_paths)
similarity_df

,data/training_image_cropped/W_01752_00_metrosexual_M.jpg,data/training_image_cropped/W_46417_70_military_W.jpg,data/training_image_cropped/W_01509_00_metrosexual_M.jpg,data/training_image_cropped/W_18951_50_feminine_W.jpg,data/training_image_cropped/W_29485_10_sportivecasual_M.jpg,data/training_image_cropped/W_01706_90_hiphop_M.jpg,data/training_image_cropped/W_07048_90_hiphop_M.jpg,data/training_image_cropped/W_02897_90_hiphop_M.jpg,data/training_image_cropped/W_10212_60_space_W.jpg,data/training_image_cropped/W_15390_60_mods_M.jpg,...,data/validation_image_cropped/W_10558_50_feminine_W.jpg,data/validation_image_cropped/W_14691_70_military_W.jpg,data/validation_image_cropped/W_05791_10_sportivecasual_W.jpg,data/validation_image_cropped/W_15662_19_normcore_M.jpg,data/validation_image_cropped/W_17742_80_bold_M.jpg,data/validation_image_cropped/T_21988_70_hippie_M.jpg,data/validation_image_cropped/W_28022_50_ivy_M.jpg,data/validation_image_cropped/W_00551_19_normcore_M.jpg,data/validation_image_cropped/W_03144_50_classic_W.jpg,data/validation_image_cropped/W_28453_10_sportivecasual_M.jpg
data/training_image_cropped/W_01752_00_metrosexual_M.jpg,1.000000,0.210899,0.398068,0.173589,0.399995,0.338813,0.182863,0.267071,0.412596,0.434329,...,0.438205,0.330292,0.484638,0.482437,0.270623,0.534438,0.350832,0.368641,0.337711,0.403886
data/training_image_cropped/W_46417_70_military_W.jpg,0.210899,1.000000,0.120637,0.247612,0.154074,0.152164,0.260764,0.244694,0.215496,0.113387,...,0.335977,0.585317,0.224921,0.275095,0.189749,0.222136,0.254618,0.184889,0.282393,0.313211
data/training_image_cropped/W_01509_00_metrosexual_M.jpg,0.398068,0.120637,1.000000,0.248750,0.611660,0.611456,0.359466,0.427839,0.331740,0.730964,...,0.212065,0.407915,0.529080,0.528548,0.424451,0.359723,0.503896,0.570038,0.362048,0.555252
data/training_image_cropped/W_18951_50_feminine_W.jpg,0.173589,0.247612,0.248750,1.000000,0.428983,0.359723,0.330876,0.293208,0.229148,0.254602,...,0.213836,0.368228,0.381539,0.298838,0.496510,0.110086,0.322565,0.383072,0.335630,0.380940
data/training_image_cropped/W_29485_10_sportivecasual_M.jpg,0.399995,0.154074,0.611660,0.428983,1.000000,0.831583,0.408877,0.370222,0.419149,0.695370,...,0.172581,0.481796,0.610824,0.599221,0.487900,0.241804,0.601420,0.714211,0.429845,0.641933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
data/validation_image_cropped/T_21988_70_hippie_M.jpg,0.534438,0.222136,0.359723,0.110086,0.241804,0.262518,0.218342,0.277995,0.365090,0.304022,...,0.355180,0.237673,0.388189,0.517182,0.310847,1.000000,0.327458,0.388033,0.335702,0.413357
data/validation_image_cropped/W_28022_50_ivy_M.jpg,0.350832,0.254618,0.503896,0.322565,0.601420,0.543252,0.369587,0.340889,0.585664,0.570041,...,0.187963,0.470272,0.467844,0.639538,0.394570,0.327458,1.000000,0.516333,0.486306,0.674002
data/validation_image_cropped/W_00551_19_normcore_M.jpg,0.368641,0.184889,0.570038,0.383072,0.714211,0.684979,0.433645,0.412945,0.315495,0.602545,...,0.162907,0.467356,0.537771,0.655204,0.491420,0.388033,0.516333,1.000000,0.437497,0.645139
data/validation_image_cropped/W_03144_50_classic_W.jpg,0.337711,0.282393,0.362048,0.335630,0.429845,0.422158,0.321874,0.259825,0.319697,0.354748,...,0.341444,0.449909,0.439299,0.496587,0.282732,0.335702,0.486306,0.437497,1.000000,0.417341


In [ ]:
# 4. 사용자 선호 데이터를 바탕으로 협업 필터링을 수행
user_data = json.load(open('user_data.json'))

# 임계값 최적화를 위한 y_true와 y_scores 초기화
y_true = []  # 실제 레이블 (1: 선호, 0: 비선호)
y_scores = []  # like_similarity - dislike_similarity 값을 기록할 리스트

DUMMY_FEATURES = feature_df.median().values.reshape(1, -1)
len(DUMMY_FEATURES)

# 선호도 예측
predictions = {}
for user_id, like_images, dislike_images, val_like_images, val_dislike_images in zip(
        user_data["R_id"], user_data["Training_Like"], user_data["Training_Dislike"],
        user_data["Validation_Like"], user_data["Validation_Dislike"]):
    
    # 각 사용자의 선호 및 비선호 이미지들의 특징 벡터
    # 만약 선호나 비선호 이미지가 없다면 더미 특징 벡터 사용
    
    valid_like_images = [img for img in like_images if img in feature_df.index]
    valid_dislike_images = [img for img in dislike_images if img in feature_df.index]
    # filtered_like_count = len(like_images) - len(valid_like_images)
    # filtered_dislike_count = len(dislike_images) - len(valid_dislike_images)
    # filtered_like_images = [img for img in like_images if img not in valid_like_images]
    # filtered_dislike_images = [img for img in dislike_images if img not in valid_dislike_images]

    # print(f"필터링된 선호 이미지 수: {filtered_like_count}, 필터링된 이미지 목록: {filtered_like_images}")
    # print(f"필터링된 비선호 이미지 수: {filtered_dislike_count}, 필터링된 이미지 목록: {filtered_dislike_images}")
    
    if not valid_like_images:
        like_features = DUMMY_FEATURES  # 더미 특징 벡터
    else:
        like_features = feature_df.loc[valid_like_images].values

    if not valid_dislike_images:
        dislike_features = DUMMY_FEATURES  # 더미 특징 벡터
    else:
        dislike_features = feature_df.loc[valid_dislike_images].values
    
    # Validation의 선호 및 비선호 이미지를 평가
    for val_image, label in zip(val_like_images + val_dislike_images, [1] * len(val_like_images) + [0] * len(val_dislike_images)):
        
        # 중앙값으로 설정
        if val_image in feature_df.index:
            try:
                val_feature = feature_df.loc[val_image].median().values.reshape(1, -1)
            except:
                val_feature = feature_df.loc[val_image].values.reshape(1, -1)
        else:
            val_feature = DUMMY_FEATURES

        like_similarity = cosine_similarity(val_feature, like_features).mean()
            
        dislike_similarity = cosine_similarity(val_feature, dislike_features).mean()

        print(f"User {user_id} - Image {val_image}: 선호 유사도 - {like_similarity:.4f}, 비선호 유사도 - {dislike_similarity:.4f}")
        
        # like_similarity - dislike_similarity 차이를 y_scores에 저장
        score_diff = like_similarity - dislike_similarity
        y_scores.append(score_diff)
        y_true.append(label)
        
        # 임계값 없이 비교해서 예측
        predicted_label = "선호" if score_diff > 0 else "비선호"
        predictions[(user_id, val_image)] = (predicted_label, "선호" if label == 1 else "비선호")

# 5. ROC 커브를 통해 최적 임계값 찾기
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
optimal_idx = np.argmax(tpr - fpr)  # tpr - fpr이 최대인 지점
optimal_threshold = thresholds[optimal_idx]
print(f"최적 임계값: {optimal_threshold}")


In [64]:
# 6. 최적 임계값을 기반으로 정확도 계산
correct = 0
total = 0

DUMMY_FEATURES = feature_df.median().values.reshape(1, -1)

for user_id, like_images, dislike_images, val_like_images, val_dislike_images in zip(
        user_data["R_id"], user_data["Training_Like"], user_data["Training_Dislike"],
        user_data["Validation_Like"], user_data["Validation_Dislike"]):
    
    # 각 사용자의 선호 및 비선호 이미지들의 특징 벡터
    # 만약 선호나 비선호 이미지가 없다면 더미 특징 벡터 사용
    valid_like_images = [img for img in like_images if img in feature_df.index]
    valid_dislike_images = [img for img in dislike_images if img in feature_df.index]
    # filtered_like_count = len(like_images) - len(valid_like_images)
    # filtered_dislike_count = len(dislike_images) - len(valid_dislike_images)
    # filtered_like_images = [img for img in like_images if img not in valid_like_images]
    # filtered_dislike_images = [img for img in dislike_images if img not in valid_dislike_images]

    # print(f"필터링된 선호 이미지 수: {filtered_like_count}, 필터링된 이미지 목록: {filtered_like_images}")
    # print(f"필터링된 비선호 이미지 수: {filtered_dislike_count}, 필터링된 이미지 목록: {filtered_dislike_images}")
    
    if not valid_like_images:
        like_features = DUMMY_FEATURES  # 더미 특징 벡터
    else:
        like_features = feature_df.loc[valid_like_images].values

    if not valid_dislike_images:
        dislike_features = DUMMY_FEATURES  # 더미 특징 벡터
    else:
        dislike_features = feature_df.loc[valid_dislike_images].values
    
    # Validation의 선호 및 비선호 이미지를 평가
    for val_image, label in zip(val_like_images + val_dislike_images, ["선호"] * len(val_like_images) + ["비선호"] * len(val_dislike_images)):        
        # 선호와 비선호 그룹 간 평균 유사도 차이를 계산
        # 중앙값으로 설정
        if val_image in feature_df.index:
            try:
                val_feature = feature_df.loc[val_image].median().values.reshape(1, -1)
            except:
                val_feature = feature_df.loc[val_image].values.reshape(1, -1)
        else:
            val_feature = DUMMY_FEATURES

        like_similarity = cosine_similarity(val_feature, like_features).mean()
            
        dislike_similarity = cosine_similarity(val_feature, dislike_features).mean()
        
        score_diff = like_similarity - dislike_similarity
        
        # 최적 임계값 적용
        final_prediction = "선호" if score_diff >= optimal_threshold else "비선호"
        print(f"User {user_id} - Image {val_image}: 예측 - {final_prediction}, 실제 - {label}")
        
        total += 1
        if final_prediction == label:
            correct += 1

# 최적 임계값 적용 후 정확도
accuracy = correct / total
print(f"\n최적 임계값 적용 후 정확도: {accuracy * 100:.2f}%")

User 12 - Image W_03412_50_classic_W: 예측 - 선호, 실제 - 선호
User 12 - Image W_02651_50_feminine_W: 예측 - 선호, 실제 - 비선호
User 27 - Image W_06522_50_ivy_M: 예측 - 선호, 실제 - 선호
User 27 - Image W_07120_19_normcore_M: 예측 - 비선호, 실제 - 선호
User 27 - Image W_17697_50_ivy_M: 예측 - 비선호, 실제 - 선호
User 30 - Image W_18249_50_feminine_W: 예측 - 선호, 실제 - 선호
User 133 - Image W_10073_70_hippie_M: 예측 - 비선호, 실제 - 비선호
User 133 - Image W_17697_50_ivy_M: 예측 - 선호, 실제 - 비선호
User 140 - Image W_07121_80_bold_M: 예측 - 비선호, 실제 - 비선호
User 179 - Image W_14147_70_disco_W: 예측 - 비선호, 실제 - 비선호
User 196 - Image W_01549_50_ivy_M: 예측 - 선호, 실제 - 선호
User 196 - Image W_12803_70_hippie_M: 예측 - 선호, 실제 - 선호
User 289 - Image W_06448_10_sportivecasual_W: 예측 - 선호, 실제 - 선호
User 289 - Image W_11495_19_genderless_W: 예측 - 비선호, 실제 - 비선호
User 368 - Image W_01703_00_metrosexual_M: 예측 - 선호, 실제 - 선호
User 368 - Image W_12817_50_ivy_M: 예측 - 선호, 실제 - 선호
User 368 - Image W_00551_19_normcore_M: 예측 - 선호, 실제 - 선호
User 368 - Image W_06864_10_sportivecasual_M: 예측 - 